# Ballistic Impact Simulation Using PyMechanical

### Objective:
Setup a model in PyMAPDL for a plate and bullet. Use it for running a Ballistic impact simulation


### Model Details

#### Model parameters:

* Thickness

## Define all parameters

In [25]:
## Main parameters
thickness = 0.004

In [26]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

# Launch MAPDL and create geometry

In [28]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [29]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('MKS')      # MKS unit system

## Create geometry
mapdl.block(0, structure_length, 0, structure_breadth, 0, thickness)

plate = mapdl.cm("PLATE", "VOLU")

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = thickness + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# Bullet
# mapdl.cone(0.0053, 0.005805, initial_distance , initial_distance + 0.00825)
mapdl.cylind(0.005805,0, initial_distance + 0.00825 , initial_distance + 0.04089 )
# mapdl.vadd(1,2)
bullet = mapdl.cm("BULLET","VOLU")

mapdl.csdele(11)                                                                   ## Delete local coordinate system
# mapdl.input('bullet9.anf')

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [31]:
# Saves the geometry in IGES format
mapdl.cdwrite("ALL",fname="C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom",ext='igs')

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
        16 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             32
        22 LINES DEFINED         MAX LINE NUMBER =                 44
        10 AREAS DEFINED         MAX AREA NUMBER =                 20
         2 VOLUMES DEFINED       MAX VOLUME NUMBER =                2
         1 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             1
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom.igs                                                                                                                                                                                                   

 WRITE IGES FILE= C:/Users/HPRVa/Ballistic_shield_design_optimization/src/

 ATTRIBUTES WIL

In [32]:
mapdl.exit()

## Launch Ansys Mechanical 

In [33]:
import json
import os
import grpc

from ansys.mechanical.core import launch_mechanical

mechanical = launch_mechanical(batch=True, cleanup_on_exit=False)
print(mechanical)

Ansys Mechanical [Ansys Mechanical Enterprise]
Product Version:231
Software build date:Wednesday, November 23, 2022 6:49:22 PM


In [36]:
output = mechanical.run_python_script(
    r"""
import json

# Import the geometry generated from the MAPDL solver
geometry_import_group_11 = Model.GeometryImportGroup
geometry_import_13 = geometry_import_group_11.AddGeometryImport()

geometry_import_13_format = Ansys.Mechanical.DataModel.Enums.GeometryImportPreference.\
    Format.Automatic
geometry_import_13_preferences = Ansys.ACT.Mechanical.Utilities.GeometryImportPreferences()
geometry_import_13_preferences.ProcessNamedSelections = True
geometry_import_13_preferences.ProcessCoordinateSystems = True
geometry_import_13.Import(r"C:\Users\HPRVa\Ballistic_shield_design_optimization\src\ geom.iges") # Primary Source!


# Specify the Unit system
ExtAPI.Application.ActiveUnitSystem = MechanicalUnitSystem.StandardMKS


# Set Stiffness behaviour of bullet to Rigid
body_25 = DataModel.GetObjectById(25)
body_25.StiffnessBehavior = StiffnessBehavior.Rigid

# Specify type of analysis
model_11 = Model
analysis_29 = model_11.AddExplicitDynamicsAnalysis()


# Specify the initial velocity of bullet
initial_conditions_34 = DataModel.GetObjectById(34)
initial_condition_37 = initial_conditions_34.InsertVelocity()
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
initial_condition_37.Location = selection
initial_condition_37.DefineBy = LoadDefineBy.Components
initial_condition_37.ZComponent = Quantity(-400, "m/s")          # Initial velocity (in z dir.) of bullet = 400 m/s

####
solution_30 = analysis_29.Solution

# Add total deformation of plate to solution
total_deformation_41 = solution_30.AddTotalDeformation()


selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [32]
total_deformation_41.Location = selection


# Add a velocity probe on the bullet ( for getting Vz)
velocity_probe_43 = solution_30.AddVelocityProbe()
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
velocity_probe_43.GeometryLocation = selection
velocity_probe_43.ResultSelection = ProbeDisplayFilter.ZAxis     # z component of velocity


# Make an energy probe on bullet
energy_probe_45 = solution_30.AddEnergyProbe()
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
energy_probe_45.GeometryLocation = selection


# Meshing controls
mesh_15 = Model.Mesh
mesh_15.PhysicsPreference = MeshPhysicsPreferenceType.Explicit   # Set mesh physics preference to explicit
mesh_15.ElementSize = Quantity(0.001, "m")                       # Set max element size
mesh_15.GenerateMesh()                                           # Generate Mesh


# Set material properties
# with Transaction(True):
#     body_22 = DataModel.GetObjectById(22)
#     body_22.Material = "cc89e716-9563-431b-98d0-b698ebff6d55"
# #endregion

# #region UI Action
# with Transaction(True):
#     body_25 = DataModel.GetObjectById(25)
#     body_25.Material = "ce7f3c8f-cb53-4f3e-9517-ac8ade5cd762"
# #endregion


# Specify End time of simulation and erosion control in analysis settings
analysis = ExtAPI.DataModel.Project.Model.Analyses[0] 
settings = analysis.AnalysisSettings
settings.ErosionOnMaterialFailureType = True
settings.SetStepEndTime(1,Quantity(1e-04,"s"))


# Solve and get the results.
# Solve Explicit Dynamics Analysis
analysis_29.Solution.Solve(True)

 """
)
print(output)

In [24]:
mechanical.exit()